<a href="https://colab.research.google.com/github/wicksonkold/Market-Sentiment-Analysis/blob/main/News_Scraping_(L).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas --upgrade --quiet
!pip install numpy --upgrade --quiet
!pip install requests-html --upgrade --quiet
!pip install pygooglenews --upgrade --quiet
!pip install transformers --upgrade --quiet
!pip install newspaper3k --upgrade --quiet
!pip install yfinance --upgrade --quiet
!pip install -U pip setuptools wheel --upgrade --quiet
!pip install -U spacy --upgrade --quiet
!pip install requests_html --upgrade --quiet
!pip install bert-extractive-summarizer


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Testing (Maybe For TR CTI)

In [2]:
'''
#CTI??? LOL
from pygooglenews import GoogleNews
from newspaper import Article
from newspaper import Config
import nltk
nltk.download('punkt')

#USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Geck/20100101 Firefox/78.0'
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.79 Safari/537.36'
config = Config()
config.browser_user_agent = USER_AGENT
config.request_timeout = 10 

gn = GoogleNews(lang = 'en', country = 'US')

def get_titlesss(search):
    stories = []
    search = gn.search(search, when = '1d') #when: duration; 2y = 2 years, 6m = 6 months, etc. 
    newsitem = search['entries']
    for item in newsitem:
        try:
          story = item.link
          url = item.link
          article = Article(url, config=config)

          article.download()
          article.parse()
          authors = ", ".join(author for author in article.authors) 
          title = article.title
          date = article.publish_date
          text = article.text
          image = article.top_image
          videos = article.movies
          url = article.url

          article.nlp()
          keywords = article.keywords
          keywords.sort()
          print("\n")
          print(item.link)
          print(f"📌 Keywords: {keywords}")
          print(f"📰 Summary: {article.summary}")
          stories.append(story)
        except:
          pass
    return stories
'''

'\n#CTI??? LOL\nfrom pygooglenews import GoogleNews\nfrom newspaper import Article\nfrom newspaper import Config\nimport nltk\nnltk.download(\'punkt\')\n\n#USER_AGENT = \'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Geck/20100101 Firefox/78.0\'\nUSER_AGENT = \'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.5112.79 Safari/537.36\'\nconfig = Config()\nconfig.browser_user_agent = USER_AGENT\nconfig.request_timeout = 10 \n\ngn = GoogleNews(lang = \'en\', country = \'US\')\n\ndef get_titlesss(search):\n    stories = []\n    search = gn.search(search, when = \'1d\') #when: duration; 2y = 2 years, 6m = 6 months, etc. \n    newsitem = search[\'entries\']\n    for item in newsitem:\n        try:\n          story = item.link\n          url = item.link\n          article = Article(url, config=config)\n\n          article.download()\n          article.parse()\n          authors = ", ".join(author for author in article.authors) \n      

In [3]:
#get_titlesss('cyber security')


#Import & Functions


In [4]:
import os
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from summarizer import Summarizer,TransformerSummarizer
from transformers import AutoModelForMaskedLM
import numpy as np
import pandas as pd
import matplotlib as plt
from newspaper import Article
import yfinance as yf
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#Scrape Google News

In [61]:
from pygooglenews import GoogleNews
#from datetime import date

def get_news(search_w):
  stories = []
  #start_date = datetime.date(2021, 1, 1)
  #end_date = datetime.date(2022,11,13)
  gn = GoogleNews(lang='en', country = 'HK')
  #search = gn.search('Hang Seng', from_=start_date.strftime('%Y-%m-%d', to_=(end_date+delta).strftime('%Y-%m-%d')))
  search = gn.search(search_w, when = '2d')
  #search = gn.search(search_w, from_ = '2022/11/21', when = '1d')
  #search = gn.geo_headlines('Hong Kong')
  #search = gn.topic_headlines('BUSINESS', proxies=None, scraping_bee=None) 
  newsitem = search['entries']
  for item in newsitem:
    story = {
        'title':item.title,
        'published':item.published,
        'published_parsed':item.published_parsed,
        'link':item.link
    }
    stories.append(story)
  return stories
'''
#Set the keyword search
test_name = get_titles('HSBC')

#append the result (Only links) into a empty list
test_link = []
for i in range(len(test_name)):
  test_link.append(test_name[i].get('link'))
print(test_link)
arc_test = test_link[0]
'''

"\n#Set the keyword search\ntest_name = get_titles('HSBC')\n\n#append the result (Only links) into a empty list\ntest_link = []\nfor i in range(len(test_name)):\n  test_link.append(test_name[i].get('link'))\nprint(test_link)\narc_test = test_link[0]\n"

#Finbert Related Function

In [26]:
#Finbert 
def sentiment_check(sentences):
  tone = []
  tone_Score = []
  esg = []
  esg_Score = []
  flsc = []
  flsc_Score = []
  esg9cat = []
  esg9cat_Score = []
  
  if type(sentences) == list:
    for idx, sent in enumerate(sentences):
      tempL, tempS = tone_A(sent)
      tone.append(tempL)
      tone_Score.append(tempS)
      '''
      tempL, tempS = esg_A(sent)
      esg.append(tempL)
      esg_Score.append(tempS)

      tempL, tempS = flsc_A(sent)
      flsc.append(tempL)
      flsc_Score.append(tempS)    

      tempL, tempS = esg9cat_A(sent)
      esg9cat.append(tempL)
      esg9cat_Score.append(tempS) 
      
    d = {'Sentences': sentences,'Tone': tone, 'Tone Score': tone_Score,
        'ESG': esg, 'ESG Score': esg_Score,
        'FLSC': flsc, 'FLSC Score': flsc_Score,
        'ESG9Cat': esg9cat, 'ESG9Cat Score': esg9cat_Score}
      '''   
  else:
    tempL, tempS = tone_A(sentences)
    tone.append(tempL)
    tone_Score.append(tempS)  

  d = {'Sentences': sentences,'Tone': tone, 'Tone Score': tone_Score}
  df_FinBERT_Score = pd.DataFrame(data=d)
  temp_df = df_FinBERT_Score
  #Calculate the score for each line and given a overall score
  sent_positive = temp_df.Tone.str.count("Positive").sum()
  sent_negative = temp_df.Tone.str.count("Negative").sum()
  sent_neutral = temp_df.Tone.str.count("Neutral").sum()
  total = sent_positive + sent_negative + sent_neutral
  
  #Weight
  sent_pov_weight = sent_positive / total
  sent_neg_weight = sent_negative / total
  sent_neut_weight = sent_neutral / total
  

  #find the largest number
  if sent_pov_weight >= sent_neg_weight and sent_pov_weight >= sent_neut_weight:
    temp_num = temp_df.loc[temp_df['Tone'] == 'Positive']
    arc_tone = ['Positive']
    arc_score = [temp_num['Tone Score'].sum()/len(temp_num['Tone Score'])]
  
  elif sent_neg_weight >= sent_pov_weight and sent_neg_weight >= sent_neut_weight:
    temp_num = temp_df.loc[temp_df['Tone'] == 'Negative']
    arc_tone = ['Negative']
    arc_score = [temp_num['Tone Score'].sum()/len(temp_num['Tone Score'])]
  
  elif sent_neut_weight >= sent_neg_weight and sent_neut_weight >= sent_pov_weight:
    temp_num = temp_df.loc[temp_df['Tone'] == 'Neutral']
    arc_tone = ['Neutral']
    arc_score = [temp_num['Tone Score'].sum()/len(temp_num['Tone Score'])]
  
  
  d1 = {'Article Tone':arc_tone, 'Article Score':arc_score}  
  df_Article_Score = pd.DataFrame(data=d1)

  return df_FinBERT_Score, df_Article_Score
  
def tone_A(sentences):
  #Tone
  finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3, cache_dir='~/from_pretrained')
  tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
  inputs = tokenizer(sentences, return_tensors="pt", padding=True)
  nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)
  try:
    results = nlp(sentences)
  except:
    firstpart = sentences[:511]
    results = nlp(firstpart)
    

  #print('Tone:',results)
  temp_label = results[0].get('label')
  temp_score = results[0].get('score')
  return temp_label, temp_score

def esg_A(sentences):
  #ESG stands for Environmental, Social, Governance or None.
  finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg',num_labels=4, cache_dir='~/from_pretrained')
  tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg')
  nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
  
  results = nlp(sentences)
  #print('ESG:',results)
  temp_label = results[0].get('label')
  temp_score = results[0].get('score')
  return temp_label, temp_score

def flsc_A(sentences):
  #Forward-looking statements
  finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-fls',num_labels=3, cache_dir='~/from_pretrained')
  tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-fls')
  nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
  
  results = nlp(sentences)
  #print('FLS:',results)# [{'label': 'Specific FLS', 'score': 0.77278733253479}]
  temp_label = results[0].get('label')
  temp_score = results[0].get('score')
  return temp_label, temp_score

def esg9cat_A(sentences):
  #Climate Change, Natural Capital, Pollution & Waste, Human Capital, Product Liability, Community Relations, Corporate Governance, Business Ethics & Values, or Non-ESG.
  finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-esg-9-categories',num_labels=9)
  tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-esg-9-categories')
  nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)

  results = nlp(sentences)
  #print('9Cat:',results) # [{'label': 'Climate Change', 'score': 0.9955655932426453}]
  temp_label = results[0].get('label')
  temp_score = results[0].get('score')
  return temp_label, temp_score 

#FinBERT Summarization

In [7]:
def finBERT_Sum(paragraph):
  model = AutoModelForMaskedLM.from_pretrained("yiyanghkust/finbert-pretrain")

  bert_model = TransformerSummarizer()
  full = ''.join(bert_model(paragraph, min_length=60))
  return full

#Newspaper3k Function - get article

In [8]:
#Newspaper3K

#this function is to clean all unwanted blank line and return a paragraph
def process_arc(url):
  sentences = (get_article(url))
  s1 = sentences.split("\n")
  temp = []
  print(s1)
  for i in range(len(s1)):
    if s1[i] != '':
      temp.append(s1[i])
  
  temp_join = ' '.join(temp)
  return temp_join

def get_article(url):
  test = ''
  try:
    #For different language newspaper refer above table
    toi_article = Article(url, language="en") # en for English
    #To download the article
    toi_article.download()
    #To parse the article
    toi_article.parse()
    #To perform natural language processing ie..nlp
    toi_article.nlp()
    '''
    #print the link
    print("Article's link:", url)
    print('==================================================')
    #To extract title
    print("Article's Title:", toi_article.title)
    print('==================================================')
    #To extract keywords
    print("Article's Keywords:", toi_article.keywords)
    print('==================================================')
    #To extract text
    print("Article's Text:", toi_article.text)
    print('==================================================')
    #To extract summary
    print("Article's Summary:", toi_article.summary)
    print('==================================================')
    '''
    test = toi_article.text
  except:
    pass

  return test


#Request_HTML


#Stocks

In [69]:
def get_stocks(name):
  # Set the start and end date
  start_date = '2020-01-01'
  end_date = '2022-11-17'

  # Set the ticker
  #ticker = ['^HSI','0005.HK','0023.HK'] 
  ticker = name 
  # Get the data
  data = yf.download(ticker, start_date, end_date)
  # Print the last 5 rows
  #print(data.head())
  df = pd.DataFrame(data=data)
  return df

#Main Body - Call Function

In [63]:
#Enter the keyword search in google news
news_list = get_news('HSBC')
print(news_list[1])


{'title': 'HSBC Campaign Helps You Cut the Strings of Financial Control - Little Black Book - LBBonline', 'published': 'Wed, 23 Nov 2022 11:00:22 GMT', 'published_parsed': time.struct_time(tm_year=2022, tm_mon=11, tm_mday=23, tm_hour=11, tm_min=0, tm_sec=22, tm_wday=2, tm_yday=327, tm_isdst=0), 'link': 'https://news.google.com/__i/rss/rd/articles/CBMiW2h0dHBzOi8vd3d3LmxiYm9ubGluZS5jb20vbmV3cy9oc2JjLWNhbXBhaWduLWhlbHBzLXlvdS1jdXQtdGhlLXN0cmluZ3Mtb2YtZmluYW5jaWFsLWNvbnRyb2zSAQA?oc=5'}


In [64]:
#append the result (Only links) into a empty list
news_title = []
news_link = []
news_published = []
for i in range(len(news_list)):
  news_title.append(news_list[i].get('title'))
  news_link.append(news_list[i].get('link'))
  news_published.append(news_list[i].get('published'))
print(news_link[:5])

['https://news.google.com/__i/rss/rd/articles/CBMif2h0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vcHJlc3MtcmVsZWFzZXMvMjAyMi0xMS0yMy9oc2JjLWJhbmstcGxjLWhzYmMtYmFuay1wbGMtZm9ybS04LTUtZXB0LXJpLWNhcGl0YWwtY291bnRpZXMtcHJvcGVydGllcy1wbGPSAQA?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMiW2h0dHBzOi8vd3d3LmxiYm9ubGluZS5jb20vbmV3cy9oc2JjLWNhbXBhaWduLWhlbHBzLXlvdS1jdXQtdGhlLXN0cmluZ3Mtb2YtZmluYW5jaWFsLWNvbnRyb2zSAQA?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMiVGh0dHBzOi8vd3d3Lmd0cmV2aWV3LmNvbS9uZXdzL29uLXRoZS1tb3ZlL2hzYmMtY3JlYXRlcy1uZXctdXMtc3RydWN0dXJlZC10cmFkZS1yb2xlL9IBAA?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMiP2h0dHBzOi8vd3d3LmZ0LmNvbS9jb250ZW50LzQ3ZmU4MTM5LTE2NzItNGQ2Yy1hMTliLWQzYTdjYWU0NzU3YtIBAA?oc=5', 'https://news.google.com/__i/rss/rd/articles/CBMibmh0dHBzOi8vd3d3LmJsb29tYmVyZy5jb20vbmV3cy9hcnRpY2xlcy8yMDIyLTExLTIzL3JibnotaGlrZS1zaWduYWxzLWdsb2JhbC10aWdodGVuaW5nLWZhci1mcm9tLW92ZXItaHNiYy1zYXlz0gEA?oc=5']


In [65]:
article_title = []
article_date = []
article_text = []
article_summarized_text = []
tone = []
tone_score = []

#create a loop to process all the news
for i in range(len(news_list)):
  sorted_arc = process_arc(news_link[i])
  article_text.append(sorted_arc)
  sum_arc = finBERT_Sum(sorted_arc)
  tempT, tempS = tone_A(sum_arc)
  
  #append to a list and transform into excel
  article_title.append(news_title[i])
  article_date.append(news_published[i])
  article_summarized_text.append(sum_arc)
  tone.append(tempT)
  tone_score.append(tempS)

['Why did this happen?', '', 'Please make sure your browser supports JavaScript and cookies and that you are not blocking them from loading. For more information you can review our Terms of Service and Cookie Policy.']


Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.pre

['Wunderman Thompson UK has launched the next phase of their integrated Financial Abuse campaign entitled ‘Control’, featuring a hard-hitting TV ad and through-the-line activity, which highlights the struggles that those trapped in economically abusive relationships face and how the bank can help.', '', 'Wunderman Thompson UK’s latest integrated campaign follows on from the out-of-home and social campaign which ran earlier this year, raising awareness that every HSBC UK branch is a ‘Safe Space’ where those experiencing domestic abuse can take their first step to financial independence by seeking specialist advice and support.', '', "Working with domestic abuse charity Hestia and its 'UK SAYS NO MORE' campaign, HSBC UK is one of the largest high street banks to participate in the Safe Spaces scheme, with specialist training to help support victims delivered to over 4,000 employees. The Safe Spaces are open to everyone, whether they are an existing HSBC UK customer or not.", '', 'The cam

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.pre

['Privacy Policy', '', 'Our privacy commitments', '', 'This Privacy Policy outlines the information we may collect about you in relation to your use of our websites, events, related publications and services (“personal data”) and how we may use that personal data. It also outlines the methods by which we and our service providers may (subject to necessary consents) monitor your online behaviour to deliver customised advertisements, marketing materials and other tailored services. This Privacy Policy also tells you how you can verify the accuracy of your personal data and how you can request that we delete or update it.', '', 'This Privacy Policy applies to all websites operated by Exporta Publishing & Events Ltd (as indicated on the relevant website).', '', 'This privacy statement does not cover the activities of third parties, and you should consult those third-party sites’ privacy policies for information on how your data is used by them.', '', 'Any questions regarding this Policy an

...

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.pre

['Agent banking outlets are emerging as a major channel for the distribution of remittance among families of migrant workers thanks to their expanding network at the rural level.', '', 'Remittance disbursement through agent banking branches grew 178 per cent to Tk 106,628 crore in the July-September quarter of the current fiscal, up from Tk 38,335 crore during the same period two years ago, showed Bangladesh Bank (BB) data.', '', '"Agent banking outlets perfectly fit the requirements of wage earners who send money from abroad to their families in rural areas. So, the distribution of remittance will grow in days to come," said Mohammad Ziaul Hasan Molla, deputy managing director of Bank Asia.', '', 'The private bank is the third largest distributor of inward remittance through its 5,400 agent banking outlets.', '', 'Bangladesh Bank said inward remittances distributed by the agent outlets increased 42.5 per cent year-on-year in the July-September quarter.', '', "The distribution of remit

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.pre

['The content you are trying to view is exclusive to our subscribers.', '', 'To unlock this article:']


Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.pre

In [40]:
print(len(article_title))
print(len(article_date))
print(len(article_summarized_text))
print(len(tone))
print(len(tone_score))
d = {'Title': article_title,
     'Date': article_date, 
     'Original Text': article_text,
     'Summarized Text': article_summarized_text,
     'FinBERT Tone': tone,
     'Tone Score': tone_score}
df_data_storage = pd.DataFrame(data=d)

100
100
100
100
100


In [62]:
df_data_storage.to_excel("output.xlsx", index = None)

df_test = pd.read_excel('output.xlsx')

df_test

,Title,Date,Original Text,Summarized Text,FinBERT Tone,Tone Score
0,HSBC Bank Plc: HSBC Bank Plc - Form 8.5 (EPT/R...,"Wed, 23 Nov 2022 10:15:33 GMT",Why did this happen? Please make sure your bro...,Please make sure your browser supports JavaScr...,Neutral,0.999831
1,RBNZ Hike Signals Global Tightening Far From O...,"Wed, 23 Nov 2022 05:52:00 GMT",Why did this happen? Please make sure your bro...,Please make sure your browser supports JavaScr...,Neutral,0.999831
2,HSBC Campaign Helps You Cut the Strings of Fin...,"Wed, 23 Nov 2022 11:00:22 GMT",Wunderman Thompson UK has launched the next ph...,Wunderman Thompson UK has launched the next ph...,Neutral,0.998732
3,January 2023 Options Now Available For HSBC Ho...,"Wed, 23 Nov 2022 16:27:00 GMT",Investors in HSBC Holdings plc (Symbol: HSBC) ...,Investors in HSBC Holdings plc (Symbol: HSBC) ...,Neutral,0.999974
4,HSBC partners with Star Alliance to offer a ne...,"Thu, 24 Nov 2022 01:44:36 GMT",The partnership between HSBC and Star Alliance...,The partnership between HSBC and Star Alliance...,Neutral,0.995569
...,...,...,...,...,...,...
95,Options Announces Ultra Low Latency 100Gb WAN ...,"Wed, 23 Nov 2022 10:32:00 GMT","Options Technology, the leading Capital Market...","Options Technology, the leading Capital Market...",Positive,0.999994
96,UK PMI Data Point to Further Weakness Ahead - ...,"Wed, 23 Nov 2022 12:38:02 GMT",UK PMI Data Point to Further Weakness Ahead 11...,UK PMI Data Point to Further Weakness Ahead 11...,Positive,0.999176
97,Menopausal women working in NHS to be offered ...,"Wed, 23 Nov 2022 12:29:20 GMT",The national NHSguidance on menopause has been...,The national NHSguidance on menopause has been...,Neutral,0.999988
98,FAQ: What is Correspondent Banking? - OCCRP,"Wed, 23 Nov 2022 17:57:12 GMT",Think of money laundering and you might imagin...,Think of money laundering and you might imagin...,Neutral,0.999919


#Enter the stocks ID and download all the data from yahoo finance

In [73]:
#0023.HK - BEA
#0005.HK - HSBC
#^HSI - Heng Seng Index
df_stock_hsbc = get_stocks('0005.HK')
df_stock_bea = get_stocks('0023.HK')
df_stock_hsi = get_stocks('^HSI')

df_stock_hsbc.to_excel("Stock HSBC.xlsx", index = None)
df_stock_bea.to_excel("Stock BEA.xlsx", index = None)
df_stock_hsi.to_excel("Stock HSI.xlsx", index = None)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


#Testing (Old Code)


In [17]:
'''
#count the pov / neg / neutral
sent_positive = df_sentences_sentiment.Tone.str.count("Positive").sum()
sent_negative = df_sentences_sentiment.Tone.str.count("Negative").sum()
sent_neutral = df_sentences_sentiment.Tone.str.count("Neutral").sum()
total = sent_positive + sent_negative + sent_neutral
print(' Positive:',sent_positive,' Ratio:',(sent_positive/total)*100,'%','\n',
   'Negative:',sent_negative,' Ratio:',(sent_negative/total)*100,'%','\n',
   'Neutral:',sent_neutral,' Ratio:',(sent_neutral/total)*100,'%','\n',
   'Total',total)
print('='*100)
d = df_sentences_sentiment
#d = df_sentences_sentiment.loc[df_sentences_sentiment['Tone'] == 'Positive']
print(d)
df_sentences_sentiment
#find the length and mean 
#print(df_sentences_sentiment['Tone Score'].sum()/len(df_sentences_sentiment['Tone Score']))
'''

'\n#count the pov / neg / neutral\nsent_positive = df_sentences_sentiment.Tone.str.count("Positive").sum()\nsent_negative = df_sentences_sentiment.Tone.str.count("Negative").sum()\nsent_neutral = df_sentences_sentiment.Tone.str.count("Neutral").sum()\ntotal = sent_positive + sent_negative + sent_neutral\nprint(\' Positive:\',sent_positive,\' Ratio:\',(sent_positive/total)*100,\'%\',\'\n\',\n   \'Negative:\',sent_negative,\' Ratio:\',(sent_negative/total)*100,\'%\',\'\n\',\n   \'Neutral:\',sent_neutral,\' Ratio:\',(sent_neutral/total)*100,\'%\',\'\n\',\n   \'Total\',total)\nprint(\'=\'*100)\nd = df_sentences_sentiment\n#d = df_sentences_sentiment.loc[df_sentences_sentiment[\'Tone\'] == \'Positive\']\nprint(d)\ndf_sentences_sentiment\n#find the length and mean \n#print(df_sentences_sentiment[\'Tone Score\'].sum()/len(df_sentences_sentiment[\'Tone Score\']))\n'